In [ ]:
#The big dataset of ultra-marathon running. Let's explore!

import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
#Load the csv file as a pandas DataFrame
df = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv')

In [ ]:
#Sneakpeak into the data
df.head()

In [ ]:
#More insights into the data
df.info()

In [ ]:
# About 7.5 million records of data. Let's refine the data to explore!

In [ ]:
#Only want data for US with 50km or 50 miles races held in 2020.

In [ ]:
# Step 1:Show 50km or 50 miles in the dataset to find the convention used for depicting distance
# Step 2: Combine 50k and 50mi using isin (or by using an 'or' statement)
# Step 3: Add the final condition year 2020
# Step 4: Filter results for USA

In [ ]:
#Getting Step 1 and Step 2 done here
df[df['Event distance/length'].isin(['50k','50mi'])]

In [ ]:
#Combining step 2 and 3
#Alternate way: df[((df['Event distance/length'] == '50km') | (df['Event distance/length'] == '50mi'))  & (df['Year of event'] == 2020)]
df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020)]

In [ ]:
#Problem Statement: There is no direct field capturing the country name. Need to extract it from the column "Event name"!

In [ ]:
df[df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA']

In [ ]:
# Combining step 2,3 and 4 to refine the final dataset for exploration

In [ ]:
df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020) & (df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA')]

In [ ]:
#Store the above obtained dataset as a new DataFrame. This is out refined data to explore!

In [ ]:
df2 = df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020) & (df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA')]

In [ ]:
df2.head(5)

In [ ]:
# We have the dataset consisting of USA only. Let's remove the redundant term (USA) from the 'Event name' column

In [ ]:
df2['Event name'] = df2['Event name'].str.split('(').str.get(0)

In [ ]:
#Clean up athlete age

In [ ]:
df2['Athlete_age'] = 2020 - df2['Athlete year of birth']

In [ ]:
df2.head()

In [ ]:
#Remove h from athlete performance

In [ ]:
df2['Athlete performance'] = df2['Athlete performance'].str.split(' ').str.get(0)

In [ ]:
df2.head()

In [ ]:
#drop columns: Athlete Club, Athlete Country, Athlete year of birth, Athlete age category

In [ ]:
df2 = df2.drop(['Athlete club','Athlete country','Athlete year of birth','Athlete age category'],axis=1)

In [ ]:
df2.head()

In [ ]:
#Clean up null values

In [ ]:
df2.isna().sum()

In [ ]:
df2[df2['Athlete_age'].isna()==1]

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

In [ ]:
#Check for duplicates

In [ ]:
df2[df2.duplicated() == True]

In [ ]:
#No duplicates found from the above code
#Reset index

In [ ]:
df2.reset_index(drop=True)

In [ ]:
#Fix few other datatypes
df2.info()

In [ ]:
#Change datatype of athlete age from float to int
df2['Athlete_age'] = df2["Athlete_age"].astype(int)

In [ ]:
#Change datatype of average speed from object to float
df2['Athlete average speed'] = df2['Athlete average speed'].astype(float)

In [ ]:
df2.head()

In [ ]:
#Rename columns with familiar convention

In [ ]:
df2 = df2.rename(columns = {'Year of event': 'year' ,
                            'Event dates': 'race_day',
                            'Event name': 'race_name',
                            'Event distance/length': 'race_length',
                            'Event number of finishers': 'race_number_of_finishers' ,
                            'Athlete performance': 'athlete_performance',
                            'Athlete gender': 'athlete_gender',
                            'Athlete average speed': 'athlete_average_speed',
                            'Athlete ID': 'athlete_id',
                            'Athlete_age': 'athlete_age',
                           })

In [ ]:
df2.head()

In [ ]:
#Reorder columns 

In [ ]:
df3 = df2[['race_day','race_name','race_length','race_number_of_finishers','athlete_id','athlete_gender','athlete_age','athlete_average_speed','athlete_performance','athlete_id']]

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
#The data is now refined as per my requirements. On we go to visualize!
#Seaborn section ahead

In [ ]:
# Distribution of 50km and 50mi races
sns.histplot(df3, x = 'race_length')

In [ ]:
# Males vs females categories
sns.histplot(df3, x = 'race_length',hue= 'athlete_gender')

In [ ]:
#Distribution of 50mi runners wrt their average speed
sns.displot(df3[df3['race_length'] == '50mi'], x = 'athlete_average_speed')

In [ ]:
#violin plot
sns.violinplot(df3,x='race_length',y='athlete_average_speed',hue='athlete_gender', split=True, inner='quarts', linewidth=1 )

In [ ]:
#lmplot
sns.lmplot(df3, x='athlete_age',y = 'athlete_average_speed', hue = 'athlete_gender')

In [ ]:
#I want to find answers/explanations for the below questions


In [ ]:
 # Difference in speed for the 50k, 50mi male to female
df3.groupby(['race_length','athlete_gender'])['athlete_average_speed'].mean()  

In [ ]:
#What age groups are the fastest in the 50mi race (min 20 races)
# df3[df3['race_length']=='50mi']
df3.query('race_length == "50mi"').groupby('athlete_age')['athlete_average_speed'].agg(['mean','count']).sort_values('mean',ascending=False).query('count >= 19')

In [ ]:
#What age groups are the slowest in the 50mi race (min 20 races)
df3.query('race_length == "50mi"').groupby('athlete_age')['athlete_average_speed'].agg(['mean','count']).sort_values('mean',ascending=True).query('count >= 20')

In [ ]:
#Season wise exploration of the data
#Spring: 3-5
#Summer: 6-8
#Fall: 9-11
#Winter: 12-2

In [ ]:
#Non month column. Need to extract month from race_day!
#df3['race_day'].str.split('.').str.get(1).astype(int)
df3['race_month'] = df3['race_day'].str.split('.').str.get(1).astype(int)

In [ ]:
df3.head()

In [ ]:
#Name a new column as race_season based on race_month

In [ ]:
#Create a function to capture season name from race_month
def my_func(x):
    if (x in [3,4,5]):
        return 'Spring'
    elif (x in [6,7,8]):
        return 'Summer'
    elif (x in [9,10,11]):
        return 'Fall'
    else:
        return 'Winter'

In [ ]:
#Add race season column using the function my_func
df3['race_season'] = df3['race_month'].apply(my_func)

In [ ]:
df3.head()

In [ ]:
df3.groupby('race_season')['athlete_average_speed'].agg(['mean','count']).sort_values('mean',ascending=False)

In [ ]:
#Interested in data of 50mi only

In [ ]:
df3.query('race_length == "50mi"').groupby('race_season')['athlete_average_speed'].agg(['mean','count']).sort_values('mean',ascending = False)